In [2]:
import numpy as np
import pandas as pd
import re
import sys
import spacy
import csv
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm, trange

'''
# i didnt have spacy or the model so I dowlaoded using this
!python -m spacy download en_core_web_md
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
'''

'\n# i didnt have spacy or the model so I dowlaoded using this\n!python -m spacy download en_core_web_md\n!{sys.executable} -m pip install spacy\n!{sys.executable} -m spacy download en\n'

In [3]:
# Credit to Sachin for this code fr thank you
df_UoA = pd.read_csv('./UoA-SDG-Keyword-List-Ver.-1.1.xlsx - SDG Keywords (initial ver.).csv')
df_UoA = df_UoA[["SDG Keywords", "Alternatives"]]
df_UoA = pd.concat([df_UoA['SDG Keywords'], df_UoA['Alternatives']], ignore_index=True)
df_UoA = df_UoA.to_frame(name='Keywords')
df_UoA.dropna(inplace=True, ignore_index=True)

df_USC = pd.read_csv('./USC-Keywords.csv')
df_USC = pd.concat([df_USC[col] for col in df_USC.columns], ignore_index=True)
df_USC = df_USC.to_frame(name='Keywords')
df_USC.dropna(inplace=True, ignore_index=True)

# combine keywords
combined_df = pd.concat([df_UoA, df_USC], ignore_index=True)
combined_df.dropna(inplace=True, ignore_index=True)
keywords_list = combined_df['Keywords'].tolist()
keywords = set(keywords_list)
keywords = sorted(list(keywords))

for i,phrase in enumerate(keywords):
    if ";" in phrase:
        segments = phrase.split(";")
        segments = [f"({segment.strip()})" for segment in segments]
        new_str = " OR ".join(segments)
        keywords[i] = new_str
    if " / " in phrase:
        segments = phrase.split(" / ")
        segments = [f"({segment.strip()})" for segment in segments]
        new_str = " OR ".join(segments)
        keywords[i] = new_str

len(keywords)

pd.DataFrame(keywords, columns=["Keywords"]).to_csv('cleaned_keywords.csv', index=False)

In [4]:
# Load spaCy model so we can try to use during matching
# 'en_core...' is just loading the english model
# nlp = spacy.load('en_core_web_md')

In [5]:
# loading elastic search
es = Elasticsearch(
    # this needs to come from the dashboard of the project itself
    cloud_id='==',  # Find this in your Elastic Cloud console
    basic_auth=('elastic', '')
)
# used the id for the place where i indexed data

In [6]:
# courses are done so now we can bulkload courses into elastic search

In [7]:
# bulk load our courses into elastic search
def index_courses(courses_csv):
    #read courses in
    courses_df = pd.read_csv(courses_csv)
    actions = [
        {
            #index our courses
            "_index": "courses",
            "_source": {
                # specify
                "abrev": row["Abrev"],
                "CourseReferenceNumber": row["courseReferenceNumber"],
                "Description": row["description"]
            }
        }
        for _, row in courses_df.iterrows()
    ]
    helpers.bulk(es, actions)
    
#lets use this for our courses!!
index_courses("courses.csv")

In [8]:
# going to try to use re module and regex to clean up data

In [12]:
# function to preprocess text
def preprocess_cleaned(text):
    text = text.lower()  
    # rmove punctuation and special characters including * and ?
    # text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'[?*]', '', text)  
    # what if we add if/else so that where space = "AND", / = "OR"? ?????????????????????????? pseduo code
    '''
    procesed = []
    keywords_df = pd.read_csv(keywords_csv)
    keywords = keywords_df['Keywords'].tolist()
    for keyword in keywords
        if '\'
            we can use regex sub and also .split() together
            https://stackoverflow.com/questions/56486564/how-to-replace-specific-words-from-entire-csv-file
            sort of like this file^
    '''
    return text

# function to load and preprocess keywords
def load_and_preprocess_keywords(keywords_csv):
    # Read keywords from csv
    keywords_df = pd.read_csv(keywords_csv)
    keywords = keywords_df['Keywords'].tolist()

    # apply preprocessing
    preprocess_cleaned_list = [preprocess_cleaned(keyword) for keyword in keywords]
    
    # save preprocessed keywords back to csv
    pd.DataFrame(preprocess_cleaned_list, columns=["Keywords"]).to_csv(keywords_csv, index=False)
    
    return preprocess_cleaned_list

# finally, let's use the function
preprocessed_keywords = load_and_preprocess_keywords('cleaned_keywords.csv')


In [25]:
''' df looks good, no * or ?, so im going to jump into vector search 
wiht the cleaned keywords and then the courses'''
df_clean = pd.read_csv('cleaned_keywords.csv')
df_clean[1200:]

,Keywords
1200,food supply chain management
1201,food supply chains or fsc
1202,food system
1203,food systems
1204,food value chain
...,...
3460,local materials
3461,mitigation
3462,per capita gdp
3463,per capita gross domestic product


In [17]:
''' in order to use vector search we need to embed the keywords
ad make sure they can be placed into/ tranfsored into numerical vectors
I will be using hugging face model to map sentences to dimensional space
https://huggingface.co/sentence-transformers/paraphrase-MiniLM-L6-v2'''

# loading model, took from huggin face. Powered by Bert :O
model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")

# vectorizing keywords
def embed_words(preprocessed_keywords):
    # set up from hugging face
    # encoding keywords
    embeddings = model.encode(preprocessed_keywords, convert_to_tensor = True)
    return embeddings

# keywords in vector
embedded_keywords = embed_words(preprocessed_keywords)

#look into normallization for embeddings **

In [18]:
# now we have to embed course descriptions
def embed_courses (courses, indexes):
    # has to be different bc courses has more than just desciptions
    pd.read_csv(courses)

In [36]:
x = embedded_keywords[1202]
y = embedded_keywords[1204]

In [37]:
import numpy as np

norm_x = np.linalg.norm(x)
norm_y = np.linalg.norm(y)

np.dot(x/norm_x, y/norm_y)

0.6142293

In [ ]:
# make a plot of all vectors using tsneat?
# llm using spacy and incorperate LLM to categorize as SDG and then 

# OR we can make a vector of SGD's and add those to the appropriate words/vector